In [31]:
import numpy as np
import scipy as sp
from scipy import linalg
from scipy import optimize
from scipy import interpolate
import sympy as sm
from scipy.special import erfinv
import pandas as pd
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
from statsmodels.discrete.discrete_model import Logit, Probit, MNLogit
import statsmodels.formula.api as smf
import warnings
warnings.filterwarnings('ignore')

In [32]:
#importing all datasets(!!please change the path before running the codes)
df_Calls = pd.read_csv(r'b. CARTIER_CALLS.csv', encoding='latin-1')
df_Clienteling = pd.read_csv(r'c. CARTIER_CLIENTELING.csv')
df_Livechat = pd.read_csv(r'd. CARTIER_LIVECHAT.csv')
df_PrevSales = pd.read_csv(r'e. CARTIER_PREVIOUS_SALES.csv')
df_Sales = pd.read_csv(r'f. CARTIER_SALES.csv')
df_Wishlist = pd.read_csv(r'g. CARTIER_WISHLIST.csv')

#concat purchase and repurchase and generate an interaction column
df_Sales['in_salesdataset']=1
df_PrevSales['in_salesdataset']=0
df_AllSales = pd.concat([df_Sales, df_PrevSales])
df_AllSales = df_AllSales.sort_values(by='ClientID', ascending= True)

df_AllSales['Calls']=np.where(df_AllSales['ClientID'].isin(df_Calls['ClientID'])==True,'yes','no')
df_AllSales['LiveChat']=np.where(df_AllSales['ClientID'].isin(df_Livechat['cLientid'])==True,'yes','no')
df_AllSales['Clienteling']=np.where(df_AllSales['ClientID'].isin(df_Clienteling['clientID'])==True,'yes','no')
df_AllSales['Wishlist']=np.where(df_AllSales['ClientID'].isin(df_Wishlist['clientID'])==True,'yes','no')

In [33]:
#Only leave "Sale" for the transaction category
data1=df_AllSales.drop(df_AllSales[(df_AllSales['TransactionCategory']!='Sale')].index)
#data1=data1.dropna(subset=['nb_days_since_last_sale'])

In [34]:
# Remove duplicates
data1.drop_duplicates(inplace=True)
data1.shape[0]

1717773

In [35]:
#View the data if needed
#data1sample=data1.head(int(round((len(df_AllSales)/(50)),0)))
#data1sample.to_csv('df_DATA1.csv')

#Collect all the needed raw data 
data2=data1[['ClientID','Gender','Channel','AgeAtTransaction','nb_days_since_last_sale',
             'TransactionDate','WeddingDate','PersonBirthDate','Calls','LiveChat','Clienteling','Wishlist', 'Turnover']]
data2=pd.DataFrame(data2)

#Pre-process the dependent variable "repurchase"
data2['repurchase'] = np.where(data2['nb_days_since_last_sale']>=1460,'yes','no')
print(data2['repurchase'].value_counts())
#Making"repurchase" into a dummy dependent variable
dummy_repurchase=pd.get_dummies(data2['repurchase'],prefix='repurchase')
data2['repurchase_yes']=dummy_repurchase['repurchase_yes']
#here repurchase is 1 and not repurchase is 0.
print(data2['repurchase_yes'].value_counts())

no     1608203
yes     109570
Name: repurchase, dtype: int64
0    1608203
1     109570
Name: repurchase_yes, dtype: int64


In [36]:
#Pre-process "Gender" as the first dummy independent variable
dummy_Gender=pd.get_dummies(data2['Gender'],prefix='Gender')
data2['Gender_Female']=dummy_Gender['Gender_Female']
#here female is 1 and male is 0.

#Pre-process "AgeAtTransaction" as a serial independent variable
data2= data2.dropna(subset=['AgeAtTransaction'])
data2['AgeAtTransaction']=data2['AgeAtTransaction'].astype(int)
data2.head()

,ClientID,Gender,Channel,AgeAtTransaction,nb_days_since_last_sale,TransactionDate,WeddingDate,PersonBirthDate,Calls,LiveChat,Clienteling,Wishlist,Turnover,repurchase,repurchase_yes,Gender_Female
122626,0011i00000UNT9LAAX,Male,Boutique,56,2441.0,2019-12-14,NaN,1964-10-13,no,no,no,no,65.678613,yes,1,0
134321,0011i00000UNT9LAAX,Male,Boutique,56,2441.0,2019-12-14,NaN,1964-10-13,no,no,no,no,81.143832,yes,1,0
125277,0011i00000UNT9LAAX,Male,Boutique,56,2441.0,2019-12-14,NaN,1964-10-13,no,no,no,no,479.618832,yes,1,0
22358,0011i00000UNT9LAAX,Male,Boutique,59,1016.0,2022-09-25,NaN,1964-10-13,no,no,no,no,8205.120000,no,0,0
110727,0011i00000UNTHbAAP,Male,Boutique,28,456.0,2021-05-11,2019-06-15,1994-11-23,no,no,yes,no,117.398943,no,0,0


In [37]:
#Pre-process "Channel" as the second dummy indepedent variable.
dummy_Channel=pd.get_dummies(data2['Channel'],prefix='Channel')
data2['Channel_Boutique']=dummy_Channel['Channel_Boutique']
data2['Channel_Web']=dummy_Channel['Channel_Web']
data2['Channel_CC']=dummy_Channel['Channel_CC']
data2.head()

,ClientID,Gender,Channel,AgeAtTransaction,nb_days_since_last_sale,TransactionDate,WeddingDate,PersonBirthDate,Calls,LiveChat,Clienteling,Wishlist,Turnover,repurchase,repurchase_yes,Gender_Female,Channel_Boutique,Channel_Web,Channel_CC
122626,0011i00000UNT9LAAX,Male,Boutique,56,2441.0,2019-12-14,NaN,1964-10-13,no,no,no,no,65.678613,yes,1,0,1,0,0
134321,0011i00000UNT9LAAX,Male,Boutique,56,2441.0,2019-12-14,NaN,1964-10-13,no,no,no,no,81.143832,yes,1,0,1,0,0
125277,0011i00000UNT9LAAX,Male,Boutique,56,2441.0,2019-12-14,NaN,1964-10-13,no,no,no,no,479.618832,yes,1,0,1,0,0
22358,0011i00000UNT9LAAX,Male,Boutique,59,1016.0,2022-09-25,NaN,1964-10-13,no,no,no,no,8205.120000,no,0,0,1,0,0
110727,0011i00000UNTHbAAP,Male,Boutique,28,456.0,2021-05-11,2019-06-15,1994-11-23,no,no,yes,no,117.398943,no,0,0,1,0,0


In [38]:
#Make variables calls, livechat, clienteling, and wishlist into dummy variables.
dummy_Calls=pd.get_dummies(data2['Calls'],prefix='Calls')
data2['Calls_yes']=dummy_Calls['Calls_yes']
dummy_LiveChat=pd.get_dummies(data2['LiveChat'],prefix='LiveChat')
data2['LiveChat_yes']=dummy_LiveChat['LiveChat_yes']
dummy_Wishlist=pd.get_dummies(data2['Wishlist'],prefix='Wishlist')
data2['Wishlist_yes']=dummy_Wishlist['Wishlist_yes']
dummy_Clienteling=pd.get_dummies(data2['Clienteling'],prefix='Clienteling')
data2['Clienteling_yes']=dummy_Clienteling['Clienteling_yes']
#Here 1 means the interaction via calls, livechat or clitenteling exists before the transaction, or the creation of wishlist exists.

In [39]:
#Process the interval of days between Transaction Date and Birthday Date, also those betwee Transaction Date and Wedding Date
data2['TransactionDate']=pd.to_datetime(data2['TransactionDate'])
data2['PersonBirthDate']=pd.to_datetime(data2['PersonBirthDate'])
data2['WeddingDate']=pd.to_datetime(data2['WeddingDate'],errors = 'coerce')

#Delete unreasonable dates
data2.loc[data2['PersonBirthDate'].dt.year <= 1900, 'PersonBirthDate']=np.nan
data2.loc[data2['WeddingDate'].dt.year >= 2030, 'WeddingDate']=np.nan
data2.loc[data2['WeddingDate'].dt.year <= 1900, 'WeddingDate']=np.nan

In [40]:
#Recode all variables to calculate date intervals for Logit model (Might need to make it an integer)
# Wedding Date Variable
data2['WeddingYear'] = data2['WeddingDate'].dt.year
data2['WeddingMonth'] = data2['WeddingDate'].dt.month
data2['WeddingDay'] = data2['WeddingDate'].dt.day
# Transaction Date Var
data2['TrxYear'] = data2['TransactionDate'].dt.year
data2['TrxMonth'] = data2['TransactionDate'].dt.month
data2['TrxDay'] = data2['TransactionDate'].dt.day
# PersonBirth Date Var
data2['BirthYear'] = data2['PersonBirthDate'].dt.year
data2['BirthMonth'] = data2['PersonBirthDate'].dt.month
data2['BirthDay'] = data2['PersonBirthDate'].dt.day
data2.head()

,ClientID,Gender,Channel,AgeAtTransaction,nb_days_since_last_sale,TransactionDate,WeddingDate,PersonBirthDate,Calls,LiveChat,...,Clienteling_yes,WeddingYear,WeddingMonth,WeddingDay,TrxYear,TrxMonth,TrxDay,BirthYear,BirthMonth,BirthDay
122626,0011i00000UNT9LAAX,Male,Boutique,56,2441.0,2019-12-14,NaT,1964-10-13,no,no,...,0,NaN,NaN,NaN,2019,12,14,1964.0,10.0,13.0
134321,0011i00000UNT9LAAX,Male,Boutique,56,2441.0,2019-12-14,NaT,1964-10-13,no,no,...,0,NaN,NaN,NaN,2019,12,14,1964.0,10.0,13.0
125277,0011i00000UNT9LAAX,Male,Boutique,56,2441.0,2019-12-14,NaT,1964-10-13,no,no,...,0,NaN,NaN,NaN,2019,12,14,1964.0,10.0,13.0
22358,0011i00000UNT9LAAX,Male,Boutique,59,1016.0,2022-09-25,NaT,1964-10-13,no,no,...,0,NaN,NaN,NaN,2022,9,25,1964.0,10.0,13.0
110727,0011i00000UNTHbAAP,Male,Boutique,28,456.0,2021-05-11,2019-06-15,1994-11-23,no,no,...,1,2019.0,6.0,15.0,2021,5,11,1994.0,11.0,23.0


In [41]:
#Calculate the days between birthday and transaction dates.
#We only take months and days into consideration because we test the occasion of wedding anniversay and birthday.
data2["Month_Diff1"]=data2[["WeddingMonth","TrxMonth"]].apply(lambda x:x["WeddingMonth"]-x["TrxMonth"],axis=1)
data2['Wed_interval']=np.where(data2["Month_Diff1"]<0,(abs(data2["Month_Diff1"]))*30+(data2['WeddingDay']-data2['TrxDay']),(abs(data2["Month_Diff1"]))*30+(data2['TrxDay']-data2['WeddingDay']))
data2['Wed_interval']=abs(data2['Wed_interval'])

data2["Month_Diff2"]=data2[["BirthMonth","TrxMonth"]].apply(lambda x:x["BirthMonth"]-x["TrxMonth"],axis=1)
data2['Birth_interval']=np.where(data2["Month_Diff2"]<0,(abs(data2["Month_Diff2"]))*30+(data2['BirthDay']-data2['TrxDay']),(abs(data2["Month_Diff2"]))*30+(data2['TrxDay']-data2['BirthDay']))
data2['Birth_interval']=abs(data2['Birth_interval'])

#Make the interval days between birthday and transaction dates, between wedding date and transaction date as third and fourth dummy independent variables
data2['Occasion_Wed'] = np.where(data2['Wed_interval']<=60,'yes','no')
data2['Occasion_Birth'] = np.where(data2['Birth_interval']<=60,'yes','no')

dummy_Occasion_Wed=pd.get_dummies(data2['Occasion_Wed'],prefix='Occasion_Wed')
data2['Occasion_Wed']=dummy_Occasion_Wed['Occasion_Wed_yes']
dummy_Occasion_Birth=pd.get_dummies(data2['Occasion_Birth'],prefix='Occasion_Birth')
data2['Occasion_Birth']=dummy_Occasion_Birth['Occasion_Birth_yes']
#here is purchased within 60 days before birthday or wedding is 1, otherwise0.

In [42]:
# WE can see mistakes in the code 

print(data2[['TransactionDate', 'PersonBirthDate', 'Occasion_Birth']].head(50))

#print(data2[['TransactionDate', 'WeddingDate', 'Occasion_Wed']].head(50))

       TransactionDate PersonBirthDate  Occasion_Birth
122626      2019-12-14      1964-10-13               1
134321      2019-12-14      1964-10-13               1
125277      2019-12-14      1964-10-13               1
22358       2022-09-25      1964-10-13               1
110727      2021-05-11      1994-11-23               0
109418      2019-02-12      1994-11-23               0
13262       2019-08-01      1994-11-23               0
117890      2019-07-02      1994-11-23               0
16553       2019-06-24      1994-11-23               0
18388       2021-05-11      1994-11-23               0
409         2019-05-08      1994-11-23               0
20910       2020-02-10      1994-11-23               0
7646        2016-12-10      1994-11-23               1
4039        2019-07-01      1994-11-23               0
20155       2021-06-15      1994-11-23               0
123825      2019-06-24      1994-11-23               0
60779       2022-05-18      1994-11-23               0
113561    

In [51]:
#ClientData = data2.groupby(['ClientID'])['Gender_Female'],'AgeAtTransaction','Channel_Boutique','Channel_Web','Channel_CC',
 #                       'Calls_yes','Clienteling_yes','LiveChat_yes','Wishlist_yes','Occasion_Wed','Occasion_Birth','intercept']]
    
ClientDataTest = data2.groupby(['ClientID'])[['repurchase_yes','Gender_Female','Channel_Boutique','AgeAtTransaction','Channel_Web','Channel_CC',
                        'Calls_yes','Clienteling_yes','LiveChat_yes','Wishlist_yes','Occasion_Wed','Occasion_Birth', 'Turnover']].max()    

In [55]:
ClientDataTest.shape[0]

216935

In [52]:
print(ClientDataTest)

                    repurchase_yes  Gender_Female  Channel_Boutique  \
ClientID                                                              
0011i00000UNT9LAAX               1              0                 1   
0011i00000UNTHbAAP               0              0                 1   
0011i00000UNTM0AAP               0              0                 1   
0011i00000UNTMyAAP               1              0                 1   
0011i00000UNTXwAAP               0              1                 1   
...                            ...            ...               ...   
0011i00001VBrioAAD               0              0                 1   
0011i00001VC38CAAT               0              0                 1   
0011i00001VcCMQAA3               0              1                 1   
0011i00001VcgKDAAZ               0              1                 0   
0011i00001Vdw67AAB               0              0                 1   

                    AgeAtTransaction  Channel_Web  Channel_CC  Calls_yes  \


In [53]:
#add constant
ClientDataTest['intercept']=1.0
Model1=ClientDataTest[['repurchase_yes','Gender_Female','AgeAtTransaction','Channel_Boutique','Channel_Web',
                    'Channel_CC','Calls_yes','LiveChat_yes','Clienteling_yes','Wishlist_yes','Turnover','intercept']]
Model2=ClientDataTest.copy()

In [54]:
#Since there are too many missing values in wedding date and birth date, we build up two models, one with them and one without.
#extract Indepedent variables into a dataframe for the model without wedding date
Model1=Model1.dropna()
Inde_var1=Model1.loc[:,['Gender_Female','AgeAtTransaction','Channel_Boutique','Channel_Web','Channel_CC',
                        'Calls_yes','Clienteling_yes','LiveChat_yes','Wishlist_yes','Turnover','intercept']]
#fit the logit model
logit1=sm.Logit(Model1['repurchase_yes'],Inde_var1)
result1=logit1.fit()
print(result1.summary())

Optimization terminated successfully.
         Current function value: inf
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:         repurchase_yes   No. Observations:               216922
Model:                          Logit   Df Residuals:                   216911
Method:                           MLE   Df Model:                           10
Date:                Fri, 30 Dec 2022   Pseudo R-squ.:                     inf
Time:                        17:39:39   Log-Likelihood:                   -inf
converged:                       True   LL-Null:                        0.0000
Covariance Type:            nonrobust   LLR p-value:                     1.000
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Gender_Female       -0.1522      0.011    -13.370      0.000      -0.175      -0.130
AgeAtTransaction